# Analyzing (RE)PPTIS simulations using an MSM approach
This notebook contains an example workflow that can be used for estimating the crossing probability and pathlengths of a (RE)PPTIS simulation.

## 1. Import the necessary functions

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import matplotlib.pyplot as plt
from pprint import pprint    # to print the vars of the pathensemble object
import numpy as np
import os
import glob

# Reading
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights
from tistools import set_tau_distrib, set_tau_first_hit_M_distrib, cross_dist_distr, pathlength_distr
from tistools import collect_tau, collect_tau1, collect_tau2, collect_taum
from tistools import ACCFLAGS, REJFLAGS

# REPPTIS analysis
from tistools import get_lmr_masks, get_generation_mask, get_flag_mask, select_with_masks
from tistools import unwrap_by_weight, running_avg_local_probs, get_local_probs, get_global_probs_from_dict, get_global_probs_from_local

# MSM functions
from tistools import construct_M
from tistools import global_pcross_msm
from tistools import mfpt_to_first_last_state, mfpt_to_absorbing_states, construct_tau_vector
from tistools import create_labels_states, print_vector, print_all_tau

inotify_add_watch(/home/elias/.config/ibus/bus/05a6c2178573356a657e0cd165380748-unix-1) failed: (No space left on device)


## 2. Load the simulation data

In [4]:

# Set the working directory
indir = "/Users/an/Documents/0_mfpt/repptis1/"  

# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_2108"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_repptismazegap2708"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/i_star/simulations/RETIS_flat_br_noswap"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_abl/paper-msm-short-term/simulations-and-analysis/trypsin-benzamidine/infrepptisanalysis/tistools-mfptanalysis/"
indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS"

# zero_minus_one = True if lambda_-1 interface is set
# zero_minus_one = False if lambda_-1 interface is not set
zero_minus_one = False

inputfile = indir + "/repptis.rst"    # When using PyRETIS, the input file for REPPTIS simulations is a .rst file
# inputfile = indir + "/retis3.rst"

# Move to working directory
os.chdir(indir)
print(os.getcwd())

# Set the ensemble folders and print them
folders = glob.glob(indir + "/0[0-9][0-9]")
folders = sorted(folders)
print(folders)

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs
['/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/002', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/003', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/004', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/005', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/006', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/007', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/

In [5]:
# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i==1:
        pe.set_in_zero_plus(True)
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)

    
    # Read order parameters order.txt/order.npy into path ensemble object, or load from order.npy file.
    # Saving order parameter files allows to speed up this notebook.
    #### CHANGE HERE ####
    pe.set_orders(load=False, acc_only=True, save=True)        # for the 1st time you run this notebook for a certain simulation, this will store .npy files
    # pe.set_orders(load=True, acc_only=True, save=False)                  # for the next times, you can read npy files (save=True/False is not important)
    # pe.set_orders(load=False, acc_only=True, save=False)     # if saving doesn't work

################################################################################
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000
pathensemble info: 
{'cyclenumbers': array([    0,     1,     2, ..., 12202, 12203, 12204]),
 'flags': array(['ACC', 'REJ', 'ACC', ..., 'REJ', 'ACC', 'REJ'], dtype='<U3'),
 'generation': array(['sh', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': True,
 'in_zero_plus': False,
 'interfaces': [[1.0, 1.0, 1.0], ['l_[0]', 'l_[0]', 'l_[0]']],
 'lambmaxs': array([1.04186, 1.04186, 1.03438, ..., 1.02932, 1.03238, 1.03238]),
 'lambmins': array([0.94194, 0.94194, 0.94659, ..., 0.22568, 0.89836, 0.89836]),
 'lengths': array([ 7,  1, 11, ...,  1, 14,  1]),
 'lmrs': array(['RMR', 'RMR', 'RMR', ..., 'RMR', 'RMR', 'RMR'], dtype='<U3'),
 'name': '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000',
 'ncycle': 12205,


## 3. Regular (RE)PPTIS analysis using tistools

### Analyze the REPPTIS simulation.

In [6]:
# Analysis output is saved to the data dictionary.
data = {}
for i, pe in enumerate(pathensembles):
    print("doing pathensemble {}".format(i))
    if i == 0:
        data[i] = {}
        continue  #  [0-] is not used for Pcross calculations

    # Classify the paths according to their path type.
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
                (pe.lmrs == ptype).astype(int), pe.weights)
    
    # Running average analysis: ["running"]
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    for (ptype, p_loc) in zip(pathtypes, 
                              running_avg_local_probs(pathtype_cycles, 
                                                      pe.weights, tr = False)):
        data[i]["running"]["plocal"][ptype] = p_loc

    # Analysis using all data: ["full"]
    plocfull = get_local_probs(pe, tr=False)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]

    # data[i] have now ["full"] and ["running"]

doing pathensemble 0
doing pathensemble 1
Weights of the different paths:
wRMR = 0
wRML = 1020
wLMR = 700
wLML = 9992
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.06546950991395435
pLML = 0.9345304900860456
Local crossing probabilities:
p2R = 0.05976775956284153
p2L = 0.9402322404371585
doing pathensemble 2
Weights of the different paths:
wRMR = 1361
wRML = 242
wLMR = 261
wLML = 3691
Local crossing probabilities:
pRMR = 0.8490330630068621
pRML = 0.15096693699313787
pLMR = 0.0660425101214575
pLML = 0.9339574898785425
Local crossing probabilities:
p2R = 0.291989198919892
p2L = 0.708010801080108
doing pathensemble 3
Weights of the different paths:
wRMR = 267
wRML = 161
wLMR = 252
wLML = 7185
Local crossing probabilities:
pRMR = 0.6238317757009346
pRML = 0.37616822429906543
pLMR = 0.033884630899556276
pLML = 0.9661153691004437
Local crossing probabilities:
p2R = 0.06598855689764781
p2L = 0.9340114431023522
doing pathensemble 4
Weights of the different paths:
wRMR = 5340
wRM

### Generate pathlength distribution figures, as in PyRETIS reports.

In [7]:
for i, pe in enumerate(pathensembles):
    upe = pe.unify_pe()
    # Pathlength distribution
    data[i]["pathlengths"] = pathlength_distr(upe)  # these might be used later or not! TODO
        
#=======================================
# make figures
makefigs = True 
if makefigs:
    for i, pe in enumerate(pathensembles):     
        if i == 0:
            continue
        # Cross distances distribution
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
        fig,ax = plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution      
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)

Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True


### Compute Pcross using in-house functions

In [8]:
# Global crossing probabilities (no error analysis)  
psfull = []
for i in range(1, len(pathensembles)):   # do not use the 0- ensemble
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})

Pminfull, Pplusfull, Pcrossfull = get_global_probs_from_dict(psfull)

In [9]:
# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(Pcrossfull, "o", c = "r")
ax.errorbar([i for i in range(len(Pcrossfull))], Pcrossfull, fmt="-o", c = "b", ecolor="r", capsize=6)


ax.set_xlabel("intf")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
ax.set_xticks(np.arange(len(interfaces)))
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")

print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
print("which is the case!")
print(Pcrossfull)
print([Pcrossfull[i]/Pcrossfull[i-1] for i in range(1,len(Pcrossfull))])
print("Here, the load immediately disappeared. For a simulation where this is")
print("not the case, the above code should be adapted a little bit.")

This should be the same as the repptis_report.pdf value: 7.983929073662175e-05
which is the case!
[1.0, 0.06546950991395435, 0.004323770771139192, 0.0008151360209500905, 0.0005219079236985182, 0.00027954889467396665, 0.0001610858994956273, 0.00010323221132130623, 8.844033475514247e-05, 8.58013397824904e-05, 8.199559321887152e-05, 7.983929073662175e-05]
[0.06546950991395435, 0.0660425101214575, 0.1885243376894666, 0.6402709612687744, 0.5356287612821318, 0.5762351508615307, 0.6408519407628752, 0.8567125863445411, 0.9701607306218543, 0.9556446720614551, 0.973702214989848]
Here, the load immediately disappeared. For a simulation where this is
not the case, the above code should be adapted a little bit.


In [10]:
# TODO DONT INCLUDE??

# Construct lists of the local probs

# Or we can use the get_global_probs_from_local function, using lists of the local probs
# These do not use the 0- ensemble
pmps = [data[i]["full"]["LMR"] for i in range(1,len(pathensembles))]
pmms = [data[i]["full"][
    "LML"] for i in range(1,len(pathensembles))]
ppps = [data[i]["full"]["RMR"] for i in range(1,len(pathensembles))]
ppms = [data[i]["full"]["RML"] for i in range(1,len(pathensembles))]
a,b,c = get_global_probs_from_local(pmps, pmms, ppps, ppms)
print("This should be the same as the repptis_report.pdf value:", c[-1])
print(c)
print([c[i]/c[i-1] for i in range(1,len(c))])

This should be the same as the repptis_report.pdf value: 7.983929073662175e-05
[1.0, 0.06546950991395435, 0.004323770771139192, 0.0008151360209500905, 0.0005219079236985182, 0.00027954889467396665, 0.0001610858994956273, 0.00010323221132130623, 8.844033475514247e-05, 8.58013397824904e-05, 8.199559321887152e-05, 7.983929073662175e-05]
[0.06546950991395435, 0.0660425101214575, 0.1885243376894666, 0.6402709612687744, 0.5356287612821318, 0.5762351508615307, 0.6408519407628752, 0.8567125863445411, 0.9701607306218543, 0.9556446720614551, 0.973702214989848]


## 4. Analysis using the MSM

### Construct transition matrix M

In [11]:
print(interfaces)
N = len(interfaces)
NS = 4*N-5
print("N", N)
# print("len pmms", len(pmms)) # TODO INCLUDE?
print("NS", NS)

labels1, labels2 = create_labels_states(N)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.5, 8.0, 10.0, 12.0, 14.0, 16.0, 19.0]
N 12
NS 43


In [12]:
print("mm", pmms)
print("mp", pmps)
print("pm", ppms)
print("pp", ppps)
print("sum", np.array(pmms)+np.array(pmps))
print("sum", np.array(ppms)+np.array(ppps))
if N > 3:  
    M = construct_M(pmms, pmps, ppms, ppps, N)
elif N == 3:
    M = construct_M_N3(pmms, pmps, ppms, ppps, N)
else:
    raise ValueError("The amount of interfaces needs to be 3 at least!")

mm [0.9345304900860456, 0.9339574898785425, 0.9661153691004437, 0.6400560224089635, 0.9636417821487111, 0.8481919083422843, 0.8549172346640701, 0.567524115755627, 0.577491961414791, 0.8216039279869067, 0.6250954441333673]
mp [0.06546950991395435, 0.0660425101214575, 0.033884630899556276, 0.3599439775910364, 0.036358217851288926, 0.15180809165771572, 0.1450827653359299, 0.432475884244373, 0.422508038585209, 0.1783960720130933, 0.37490455586663274]
pm [1.0, 0.15096693699313787, 0.37616822429906543, 0.0582010582010582, 0.27313432835820894, 0.19036046982584043, 0.30338983050847457, 0.0891271056661562, 0.1950427854824432, 0.30004342162396874, 0.4691265060240964]
pp [0.0, 0.8490330630068621, 0.6238317757009346, 0.9417989417989417, 0.7268656716417911, 0.8096395301741596, 0.6966101694915254, 0.9108728943338438, 0.8049572145175568, 0.6999565783760313, 0.5308734939759037]
sum [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
sum [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [13]:
# We can print the transition matrix M and check that all rows sum to 1.
print("M")
print("shape", M.shape)
print("sum prob in rows", np.sum(M,axis=1))
print(M)

M
shape (43, 43)
sum prob in rows [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.         0.93453049 0.06546951 ... 0.         0.         0.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]]


### Look at this Markov model
*INCLUDE?*

In [14]:
#import numpy.linalg
vals, vecs = np.linalg.eig(M)
print(vals)
vals, vecs = np.linalg.eig(M.T)
print(vals)
pprint(M)

[ 1.00000000e+00+0.00000000e+00j  9.84351934e-01+0.00000000e+00j
  9.60452257e-01+0.00000000e+00j  9.10253951e-01+0.00000000e+00j
  8.92756223e-01+0.00000000e+00j  7.32260170e-01+0.00000000e+00j
  6.72669413e-01+0.00000000e+00j  5.01404447e-01+0.00000000e+00j
  5.76920344e-01+0.00000000e+00j  6.01156252e-01+0.00000000e+00j
  2.69946326e-13+2.46412793e-01j  2.69946326e-13-2.46412793e-01j
 -9.99927659e-01+0.00000000e+00j -9.84555947e-01+0.00000000e+00j
 -9.60231016e-01+0.00000000e+00j -9.10513032e-01+0.00000000e+00j
 -8.92581203e-01+0.00000000e+00j -5.01404561e-01+0.00000000e+00j
 -7.32274431e-01+0.00000000e+00j -6.72656073e-01+0.00000000e+00j
 -5.76915405e-01+0.00000000e+00j -6.01165663e-01+0.00000000e+00j
 -4.04274138e-16+5.95029906e-09j -4.04274138e-16-5.95029906e-09j
  5.67784005e-16+0.00000000e+00j  2.81790385e-16+0.00000000e+00j
 -1.73319401e-16+0.00000000e+00j  1.17907456e-16+0.00000000e+00j
  4.24310299e-17+7.19305161e-17j  4.24310299e-17-7.19305161e-17j
 -7.71267113e-17+2.755953

In [15]:
print("what if chain propagates")
print("A[0,:]")
# check stationary behavior
A = M
for n in range(10):
    A = np.dot(A,M)
    #print(A)
    print(A[0,:])
    print(np.sum(A[0,:]))  # is 1 indeed

what if chain propagates
A[0,:]
[0.93453049 0.         0.         0.         0.06114574 0.00432377
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
1.0
[0.00000000e+00 8.73347237e-01 6.11832532e-02 6.11457391e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.17726139e-03 1.46509377e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e

### Pcross with MSM

In [16]:
# Inspect Z and Y vectors

z1, z2, y1, y2 = global_pcross_msm(M)
print("Z")
print_vector(z1, labels1)
print_vector(z2, labels2)
print("Y")
print_vector(y1, labels1)
print_vector(y2, labels2)
print("\nGlobal crossing probability: ", y1[0][0])

Z
state 0-     : 0
state B      : 1
state 0+- LML: 0.0
state 0+- LMR: 0.0012194881379370916
state 0+- RML: 0.0
state 1+- LML: 0.0
state 1+- LMR: 0.018465199697806075
state 1+- RML: 0.0
state 1+- RMR: 0.018465199697806075
state 2+- LML: 0.015677565058461684
state 2+- LMR: 0.0979459730457801
state 2+- RML: 0.015677565058461677
state 2+- RMR: 0.0979459730457801
state 3+- LML: 0.06699921209727948
state 3+- LMR: 0.15297581644447575
state 3+- RML: 0.06699921209727946
state 3+- RMR: 0.15297581644447575
state 4+- LML: 0.14797188709093523
state 4+- LMR: 0.2856004522204858
state 4+- RML: 0.1479718870909352
state 4+- RMR: 0.2856004522204858
state 5+- LML: 0.24800936652092198
state 5+- LMR: 0.4956317777446997
state 5+- RML: 0.24800936652092198
state 5+- RMR: 0.4956317777446997
state 6+- LML: 0.4484942592047339
state 6+- LMR: 0.7733951420271826
state 6+- RML: 0.4484942592047339
state 6+- RMR: 0.7733951420271826
state 7+- LML: 0.6748235182556261
state 7+- LMR: 0.9027474958984074
state 7+- RML: 0.674

### Pathlength analysis

In [17]:
# Setting path ensemble properties
#==================================
for i,fol in enumerate(folders):
    print(i)
    print("Calculating path lengths.")
    set_tau_distrib(pathensembles[i])
    print("Done.")

    if True:
        print("Calculating first hitting lengths to middle interface")
        set_tau_first_hit_M_distrib(pathensembles[i])
        print("Done.")

0
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
1
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
2
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
3
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
4
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
5
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
6
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
7
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
8
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
9
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
10
Calculating path lengths.
Done.
Calculating first hitting

In [18]:
# Additional information
#==================================
# Average path lengths per ensemble for each path type
print(indir[-20:])
pathtypes = ("LML", "LMR", "RMR", "RML", "LM*", "*M*", "***", "RM*", "L**", "**R", "R**")

print("=" * 80)
print("AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE")
print("=" * 80)

for i, pe in enumerate(pathensembles):
    print(f"\nEnsemble {i} ({pe.name}):")
    print("-" * 50)
    
    # Get accepted paths only
    accepted_mask = np.isin(pe.flags, ACCFLAGS)
    total_accepted_count = np.sum(accepted_mask)
    
    if total_accepted_count > 0:
        # Calculate weighted average for all accepted paths
        accepted_lengths = pe.lengths[accepted_mask]
        accepted_weights = pe.weights[accepted_mask]
        total_weighted_avg = np.average(accepted_lengths, weights=accepted_weights)
        
        print(f"  All accepted paths: {total_weighted_avg:8.2f} (n={total_accepted_count:4d}, weighted)")
        print("-" * 30)
    
    for ptype in pathtypes:
        mask = (pe.lmrs == ptype) & accepted_mask
        if np.any(mask):
            lengths = pe.lengths[mask]
            weights = pe.weights[mask]
            weighted_avg = np.average(lengths, weights=weights)
            count = np.sum(mask)
            print(f"  {ptype:4s}: {weighted_avg:8.2f} (n={count:4d}, weighted)")
        else:
            print(f"  {ptype:4s}: {0:8.2f} (n={0:4d}, weighted)")

s_data_sc10_12intfs/
AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE

Ensemble 0 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000):
--------------------------------------------------
  All accepted paths:    63.09 (n=2467, weighted)
------------------------------
  LML :     0.00 (n=   0, weighted)
  LMR :     0.00 (n=   0, weighted)
  RMR :    63.09 (n=2467, weighted)
  RML :     0.00 (n=   0, weighted)
  LM* :     0.00 (n=   0, weighted)
  *M* :     0.00 (n=   0, weighted)
  *** :     0.00 (n=   0, weighted)
  RM* :     0.00 (n=   0, weighted)
  L** :     0.00 (n=   0, weighted)
  **R :     0.00 (n=   0, weighted)
  R** :     0.00 (n=   0, weighted)

Ensemble 1 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001):
--------------------------------------------------
  All accepted paths:    65.45 (n=2512, weighted)
------------------------------
  LML :    41.99 (n=2287, weighted

In [19]:
# Compute taus for pathlength analysis
tau_mm, tau_mp, tau_pm, tau_pp = collect_tau(pathensembles)
tau1_mm, tau1_mp, tau1_pm, tau1_pp = collect_tau1(pathensembles)
tau2_mm, tau2_mp, tau2_pm, tau2_pp = collect_tau2(pathensembles)
taum_mm, taum_mp, taum_pm, taum_pp = collect_taum(pathensembles)

Collect tau
ensemble 0 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000
ensemble 1 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001
ensemble 2 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/002
ensemble 3 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/003
ensemble 4 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/004
ensemble 5 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/005
ensemble 6 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/006
ensemble 7 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/007
ensemble 8 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTI

In [20]:
# Look at computed taus
print("tau")
print_all_tau(pathensembles, tau_mm, tau_mp, tau_pm, tau_pp)
print("\ntau1")
print_all_tau(pathensembles, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
print("\ntaum")
print_all_tau(pathensembles, taum_mm, taum_mp, taum_pm, taum_pp)
print("\ntau2")
print_all_tau(pathensembles, tau2_mm, tau2_mp, tau2_pm, tau2_pp)

tau
Index Name            mm           mp           pm           pp
-----------------------------------------------------
0     000            nan          nan          nan         61.1
1     001           40.0        272.0        150.1          nan
2     002         1721.0       5837.3      12498.0       2861.6
3     003         1047.0       2439.8       2869.2       3328.3
4     004         3225.3       4231.3       2765.7       1355.2
5     005         1124.7       3268.1       1753.8       1785.9
6     006         1888.9       7217.7       5461.4       2506.0
7     007        11041.5      28589.6      30788.1      10411.6
8     008         5903.8       7140.7       6446.3       3452.7
9     009         2386.6       3464.3       2950.9       2837.8
10    010         2995.8       1909.0       1778.3       1966.8
11    011         1900.3       2041.1       2020.3       2198.3

tau1
Index Name            mm           mp           pm           pp
----------------------------------------

In [21]:
# TODO include prints?
tau  = construct_tau_vector(N, NS, tau_mm, tau_mp, tau_pm, tau_pp)
tau1 = construct_tau_vector(N, NS, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
taum = construct_tau_vector(N, NS, taum_mm, taum_mp, taum_pm, taum_pp)
tau2 = construct_tau_vector(N, NS, tau2_mm, tau2_mp, tau2_pm, tau2_pp)
tau_m = tau-tau1-tau2  # yes, this is the same thing as taum

print("tau")
print(tau)
print("\n")
print("tau1")
print(tau1)
print("taum")
print(taum)
print("tau2")
print(tau2)

print("\n")
print("tau = tau1+taum+tau2 => difference is", np.sum((tau-tau1-taum-tau2)**2))

tau
[   61.08603032    39.98829063   272.00857143   150.1
  1721.03630452  5837.29118774 12497.97933884  2861.60249816
  1046.99234516  2439.75396825  2869.21118012  3328.3258427
  3225.25164114  4231.31128405  2765.66666667  1355.22734082
  1124.69553116  3268.10245902  1753.79234973  1785.91581109
  1888.91768679  7217.73349057  5461.42765957  2506.01350675
 11041.54669704 28589.63758389 30788.11731844 10411.56934307
  5903.7733711   7140.74349442  6446.27835052  3452.66106254
  2386.6091314   3464.28614916  2950.86384266  2837.80791789
  2995.79482072  1909.04587156  1778.32416787  1966.79776675
  1900.26302932  2041.07264087     0.        ]


tau1
[   0.            0.            0.          150.1         175.68843132
  200.61685824  199.54132231  280.47391624  237.89533751  208.93253968
  144.9378882   508.31460674  214.32166302  190.68871595  185.84242424
  239.6335206   195.61094789  190.91393443  253.42076503  404.37577002
  413.00928662  515.75707547  263.17659574  367.83841921

## 5. Flux calculation

### Collect tau for [0+]

In [22]:
# Construct g and h vectors
g1, g2, h1, h2 = mfpt_to_first_last_state(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2)) #, doprint=True)
print("G")
print_vector(g1, labels1)
print_vector(g2, labels2)
print("H")
print_vector(h1, labels1)
print_vector(h2, labels2)
print("\ntau [0+]: ", h1[0])

[ 0 42] g1:  [[0.]
 [0.]] D [[1. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]] tp [[   39.98829063]
 [  272.00857143]
 [    0.        ]
 [ 1545.34787321]
 [ 5636.6743295 ]
 [12298.43801653]
 [ 2581.12858193]
 [  809.09700765]
 [ 2230.82142857]
 [ 2724.27329193]
 [ 2820.01123596]
 [ 3010.92997812]
 [ 4040.62256809]
 [ 2579.82424242]
 [ 1115.59382022]
 [  929.08458327]
 [ 3077.18852459]
 [ 1500.3715847 ]
 [ 1381.54004107]
 [ 1475.90840017]
 [ 6701.97641509]
 [ 5198.25106383]
 [ 2138.17508754]
 [10429.71981777]
 [27928.02013423]
 [29566.46927374]
 [ 9350.88077859]
 [ 4963.14164306]
 [ 5499.43494424]
 [ 6047.97250859]
 [ 2909.06792199]
 [ 1844.8752784 ]
 [ 2832.2739726 ]
 [ 24

### The flux

In [23]:
flux = 1/(tau[0]+h1[0][0])
dt = 0.002 # Change if needed
sc = 10
flux /= (dt*sc)
# print(flux/(dt*sc), "1/time")
print(flux, "1/ps")

0.11167211047744394 1/ps


## 6. The rate constant
We can compute an accurate rate constant using only our MSM.

In [24]:
# rate constant = flux * Pcross

print("The rate constant k is: ", flux*y1[0][0], "1/ps")

The rate constant k is:  8.915822095580835e-06 1/ps


## 7. Direct rate computation via $\tau_{\mathcal{A},1}$ 

In [25]:
# Construct g and h vectors
absor = np.array([NS - 1])
kept = np.array([i for i in range(NS) if i not in absor])

g1, g2, h1, h2 = mfpt_to_absorbing_states(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2), absor, kept, remove_initial_m=False, doprint=True)
print("G")
print_vector(g1, labels1[-1])
print_vector(g2, [labels1[0]] + labels2)
print("H")
print_vector(h1, labels1[-1])
print_vector(h2, [labels1[0]] + labels2)
print("interesting")
print(h2[0])
mfpt = h2[0][0]  # tau_A,1

[42] g1:  [[0.]] D [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]] tp [[   61.08603032]
 [   39.98829063]
 [  272.00857143]
 [    0.        ]
 [ 1545.34787321]
 [ 5636.6743295 ]
 [12298.43801653]
 [ 2581.12858193]
 [  809.09700765]
 [ 2230.82142857]
 [ 2724.27329193]
 [ 2820.01123596]
 [ 3010.92997812]
 [ 4040.62256809]
 [ 2579.82424242]
 [ 1115.59382022]
 [  929.08458327]
 [ 3077.18852459]
 [ 1500.3715847 ]
 [ 1381.54004107]
 [ 1475.90840017]
 [ 6701.97641509]
 [ 5198.25106383]
 [ 2138.17508754]
 [10429.71981777]
 [27928.02013423]
 [29566.46927374]
 [ 9350.88077859]
 [ 4963.14164306]
 [ 5499.43494424]
 [ 6047.97250859]
 [ 2909.06792199]
 [ 1844.8752784 ]
 [ 2832.2739726 ]
 [ 2479.57186082]
 [ 2308.52272727]
 [ 2362.21513944]
 [ 1539.93944954]
 [ 1548.18668596]
 [ 1592.81699752]
 [ 14

In [26]:
k_flux_pcross = flux * y1[0][0]
k_mfpt = 1 / (mfpt*dt*sc)

print(f"Rate constant from P_cross × flux: {k_flux_pcross:.10e} [1/ps]")
print(f"Rate constant from MFPT:           {k_mfpt:.10e} [1/ps]")
print(f"Relative difference:               {abs(k_flux_pcross - k_mfpt)/k_mfpt*100:.2f}%")


Rate constant from P_cross × flux: 8.9158220956e-06 [1/ps]
Rate constant from MFPT:           8.9158220956e-06 [1/ps]
Relative difference:               0.00%
